In [18]:
import re
import torch
import torch.nn as nn
import json
from nepalikit.tokenization import Tokenizer
from nepalikit.preprocessing import TextProcessor
from nepalikit.manage_stopwords import load_stopwords

In [6]:
class CustomLSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, dropout=0.5):
        super(CustomLSTMClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=2, batch_first=True, dropout=dropout, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids, attention_mask):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)
        pooled_output = torch.mean(lstm_out, dim=1)
        dropped_out = self.dropout(pooled_output)
        return self.fc(dropped_out)

In [7]:
with open('/home/pranjal/Downloads/fuse_project/tokenizer_vocab.json', 'r', encoding='utf-8') as vocab_file:
    vocab = json.load(vocab_file)

In [8]:
tokenizer = Tokenizer()

class CustomTokenizer:
    def __init__(self, tokenizer, vocab=None):
        self.tokenizer = tokenizer
        self.vocab = vocab if vocab else {}

    def build_vocab(self, texts):
        index = 2
        self.vocab['<PAD>'] = 0
        self.vocab['<UNK>'] = 1
        for text in texts:
            tokens = self.tokenizer.tokenize(text, level = 'word')
            for token in tokens:
                if token not in self.vocab:
                    self.vocab[token] = index
                    index += 1

    def encode(self, text, max_length):
        tokens = self.tokenizer.tokenize(text)
        token_ids = [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]
        token_ids = token_ids[:max_length] + [self.vocab['<PAD>']] * (max_length - len(token_ids))
        return token_ids
    
custom_tokenizer = CustomTokenizer(tokenizer, vocab=vocab)

In [9]:
num_classes = 4
vocab_size = 31159
max_length = 256

In [10]:
model = CustomLSTMClassifier(vocab_size=vocab_size, embedding_dim=25, hidden_dim=8, output_dim=num_classes)
model.load_state_dict(torch.load('/home/pranjal/Downloads/fuse_project/nepali_text_classification_model.pth', weights_only=True))

<All keys matched successfully>

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_names = ['Education', 'ID', 'Policy', 'Press_Release']

In [19]:
processor = TextProcessor()

stopwords_path = '/home/pranjal/Downloads/fuse_project/NepaliStopWords'
stopwords = load_stopwords(stopwords_path)

def preprocess_text(text):

    text = processor.remove_html_tags(text)
    text = processor.remove_special_characters(text)
    text = re.sub('\n', ' ', text)
    text = re.sub(r'[\d०१२३४५६७८९]', '', text)
    text = re.sub(r'\b[a-zA-Z]+\b', '', text)
    words = text.split()
    filtered_words = [word for word in words if word not in stopwords]
    filtered_words = [word for word in filtered_words if len(word) > 2]
    return ' '.join(filtered_words)

In [20]:
model.to(device)
model.eval()

def predict(text, tokenizer, model, max_length, device):
    text = preprocess_text(text)
    model.eval()
    with torch.no_grad():
        encoded_input = tokenizer.encode(text, max_length)
        input_ids = torch.tensor(encoded_input, dtype=torch.long).unsqueeze(0).to(device)
        attention_mask = (input_ids != tokenizer.vocab['<PAD>']).long().to(device)

        outputs = model(input_ids, attention_mask)
        _, predicted_class = torch.max(outputs, 1)

        return predicted_class.item()

# sample_text = "बल्खु त्रिभुवन विश्वविद्यालय विज्ञान प्रविधि अध्ययन संस्थान डीनको कार्यालय परीक्षा शाखा वल्खुद्वारा सेमेष्टर परीक्षा प्रणाली साल चैत्र महिनामा संचालन वर्षे स्नातक कम्प्यूटर विज्ञान सुचना प्रविधि विषय वर्ष शत्र समूहको नियमित शकत्रको अगाडिका समूहको आंशिक परीक्षाको परीक्षाफल विज्ञान प्रविधि अध्ययन संस्थान परीक्षा संचालक समितिको निर्णयानुसार प्रकाशित गरिएको परीक्षामा निम्नाकित कमांक परीक्षार्थीहरु उत्तीर्ण वर्षे स्नातक कम्प्यूटर विज्ञान सूचना प्रविधि वर्ष शत्र"
sample_text = "ग्र्ट उपनियम वमोजिम जम्मा रकम सम्वन्धित ठेकेदार आपूर्तिकर्ताले बुझाउनु करमा मिलान पाउनेछ कारोबार गराउनु विशेष व्यक्तिको वार्षिक कारोबार वस्तु ढुवानी साधन भाडा ढुवानी सेवाको हकमा पतच्रास रुपैयाँसम्म बस्तु सेवा भएमा खुलाइ अनुसूची बमोजिमको ढाँचामा कारोबार गर्नको सम्बन्धित अधिकृतसमक्ष दरखास्त दिनु पर्नेछ उपनियम बमोजिम अनुमान सक्रिने नभे रुपैयाँसम्म वस्तु सेवा मिश्रित कारोबार सेवा व्यवसायको हक्रमा वीस रुपैयाँ वमोजिमको ढाँचामा कारोतार गर्नको सम्बन्धित क्रर अधिकृत समक्ष दरखास्त दिनु पर्नेछ ऐनको दफा उपदफा व्रमोजिम गर्नुपर्ने वस्तु सेवाको पर्नेछ अधिकृतले निरीक्षण गर्दाका बखत कारोबार व्यक्तिसँग आटौं संशोधनद्वारा नवौं संशोधनद्वारा संशोधित"
predicted_label = predict(sample_text, custom_tokenizer, model, max_length, device)
predicted_class_name = class_names[predicted_label]
print(f"Predicted class: {predicted_class_name}")

Predicted class: Press_Release
